In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from tqdm import tqdm
import seaborn as sns
import re
import datetime
import os
import random
import h3 
import shap 

In [171]:
import json

h3_l7_df = pd.read_json("h3_l7_df.json", orient="records", lines=True)

In [172]:
h3_l7_df

,id,school,parking,grave_yard,college,hospital,fuel,place_of_worship,fast_food,stage,...,car_rental,driving_school,payment_centre,dojo,gambling,bar,bicycle_parking,bbq,social_facility,ohca
0,872af0424ffffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,872af0425ffffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,872af0426ffffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,872af0500ffffff,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,872af0501ffffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,872af6373ffffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
133,872af6375ffffff,0,4,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,17
134,872af6acaffffff,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
135,872af6ad9ffffff,0,53,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,2


In [173]:
# 設定plt環境
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [174]:
# # 設定訓練、驗證和測試集的比例
# train_proportion = 0.7  

# train_size = int(h3_l7_df.shape[0]*train_proportion)

# seed = 7578 #666
# torch.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)

# # h3_l7_id = np.random.choice(spatial_data.shape[0], spatial_data.shape[0])
# # spatial_data = spatial_data[h3_l7_id]
# # train_spatial_data = spatial_data[:train_size, :]
# # test_spatial_data = spatial_data[train_size:, :]

# # 隨機打亂 DataFrame
# shuffled_h3_l7_df = h3_l7_df.sample(frac=1, random_state=seed).reset_index(drop=True)

# # 分割訓練集和測試集
# train_h3_l7_df = shuffled_h3_l7_df.iloc[:train_size]
# test_h3_l7_df = shuffled_h3_l7_df.iloc[train_size:]

In [175]:
# # 將 h3_l7_df 資料框中的 'id' 列移除，僅保留數據進行正規化
# shuffled_spatial_data = shuffled_h3_l7_df.drop('id', axis=1).drop('police', axis=1)


# # # # 對數據進行正規化：將每個數據列的最小值調整為 0，最大值調整為 1
# normalized_spatial_data = (shuffled_spatial_data - shuffled_spatial_data.min()) / (shuffled_spatial_data.max() - shuffled_spatial_data.min())

# #設定OHCA正規化反函數 方便把預測結果返回原本scale
# ohca_reguli_inverse = (h3_l7_df.ohca.max()-h3_l7_df.ohca.min()) + h3_l7_df.ohca.min()

# # 將 DataFrame 轉換為 numpy array，並設定數據類型為 np.float64
# spatial_data = np.array(normalized_spatial_data).astype(np.float64)


# train_spatial_data = spatial_data[:train_size, :]
# test_spatial_data = spatial_data[train_size:, :]

In [176]:
train_index=[]
test_index=[]
for i in range(0,h3_l7_df.shape[0]):
    geo_location = h3.h3_to_geo(h3_l7_df.iloc[i]['id'])
    if (geo_location[1]) > (-76.10):
        train_index.append(i)
    else:
        test_index.append(i)

# 分割訓練集和測試集
train_h3_l7_df = h3_l7_df.iloc[train_index]
test_h3_l7_df = h3_l7_df.iloc[test_index]
print(len(train_index))
print(len(test_index))


92
45


In [177]:
# 將 h3_l7_df 資料框中的 'id' 列移除，僅保留數據進行正規化
h3_spatial_data = h3_l7_df.drop('id', axis=1).drop('police', axis=1)


# # # 對數據進行正規化：將每個數據列的最小值調整為 0，最大值調整為 1
normalized_spatial_data = (h3_spatial_data - h3_spatial_data.min()) / (h3_spatial_data.max() - h3_spatial_data.min())

#設定OHCA正規化反函數 方便把預測結果返回原本scale
ohca_reguli_inverse = (h3_l7_df.ohca.max()-h3_l7_df.ohca.min()) + h3_l7_df.ohca.min()

# 將 DataFrame 轉換為 numpy array，並設定數據類型為 np.float64
spatial_data = np.array(normalized_spatial_data).astype(np.float64)


train_spatial_data = spatial_data[train_index]
test_spatial_data = spatial_data[test_index]


In [178]:
class Regressor(nn.Module):
    """
    用於迴歸任務的神經網絡模型 Regressor。
    
    結構:
    - 兩層隱藏層，並使用 ReLU 激活函數
    - 最後一層為線性層，不使用激活函數（適用於迴歸）
    """
    def __init__(self, input_size=2, hidden_size=32, output_size=1):
        super().__init__()
        # 定義三層全連接層
        self.fc1 = nn.Linear(input_size, hidden_size)     # 第一層：輸入層到隱藏層
        self.fc2 = nn.Linear(hidden_size, hidden_size)    # 第二層：隱藏層到隱藏層
        self.fc3 = nn.Linear(hidden_size, output_size)    # 第三層：隱藏層到輸出層

        # 初始化權重和偏置
        nn.init.normal_(self.fc1.weight, std=0.02)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.normal_(self.fc2.weight, std=0.02)
        nn.init.constant_(self.fc2.bias, 0)
        nn.init.normal_(self.fc3.weight, std=0.02)
        nn.init.constant_(self.fc3.bias, 0)
        
    def forward(self, input):
        # 前向傳播過程
        output = F.relu(self.fc1(input))  # 第一層 + ReLU 激活
        output = F.relu(self.fc2(output)) # 第二層 + ReLU 激活
        output = self.fc3(output)         # 第三層（不使用激活函數）
        return output

In [179]:
window_size = 1

def train_reg(spatial_data, 
              s_net,
              s_net_optim, 
              window_size, iter_num=5000):
    """
    訓練 s_net  網絡來預測 spatial_data 中的數據。
    
    參數:
    - spatial_data: numpy array，包含訓練數據
    - s_net: 神經網絡模型
    - s_net_optim: 優化器
    - window_size: 每次迭代的隨機取樣大小
    - iter_num: 訓練迭代次數
    
    返回:
    - loss_array: 每次迭代的損失值
    - t_fea_array, s_fea_array: 用於存儲特徵的暫時性陣列（目前未使用）
    """

    loss_array = []     # 儲存每次迭代的損失
    t_fea_array = []    # 預留用於儲存暫時性特徵的空列表
    s_fea_array = []    # 預留用於儲存暫時性特徵的空列表

    for _ in tqdm(range(iter_num)):
        
        # 隨機選擇一組數據索引
        h3_l7_id = np.random.choice(spatial_data.shape[0] - 1, window_size)

        # s_net 輸入特徵向量並計算輸出
        # s_fea = s_net(torch.autograd.Variable(torch.FloatTensor(spatial_data[h3_l7_id, :-1])))

        # 提取目標變數（即輸入的最後一列數據）並轉為 Tensor
        ohca = spatial_data[h3_l7_id, -1].reshape(-1, 1)
        ohca = torch.autograd.Variable(torch.FloatTensor(ohca))

        # p_pred 用於預測目標變數
        p_pred = s_net(torch.autograd.Variable(torch.FloatTensor(spatial_data[h3_l7_id, :-1]))).reshape(-1, 1)

        # 定義均方誤差損失
        mseloss = torch.nn.MSELoss(reduction='sum')
        loss = mseloss(p_pred, ohca)
        
        # 清空前一次計算的梯度
        s_net_optim.zero_grad()
        
        
        # 計算損失的梯度
        # loss.backward()
        autograd.backward(loss)

        # 更新神經網絡參數
        s_net_optim.step()
        
        # 儲存損失值
        loss_array.append(loss.detach().cpu().numpy())

    return loss_array, t_fea_array, s_fea_array

# 初始化模型和優化器
s_net = Regressor(input_size=spatial_data.shape[1] - 1, hidden_size=spatial_data.shape[1] * 2, output_size=1)
s_net_optim = optim.Adam(s_net.parameters(), lr=1e-3, weight_decay=1e-5)

iter_num=30000
# 執行訓練過程
loss_array, t_fea_array, s_fea_array = train_reg(train_spatial_data, s_net,
                                                 s_net_optim,
                                                 window_size, iter_num)

  0%|          | 0/30000 [00:00<?, ?it/s]

100%|██████████| 30000/30000 [00:50<00:00, 591.74it/s]


In [180]:
y_head_train = s_net(torch.autograd.Variable(torch.FloatTensor(train_spatial_data[:, :-1]))).detach().numpy()*ohca_reguli_inverse
y_train = train_spatial_data[:, -1]*ohca_reguli_inverse
y_head_test = s_net(torch.autograd.Variable(torch.FloatTensor(test_spatial_data[:, :-1]))).detach().numpy()*ohca_reguli_inverse
y_test = test_spatial_data[:, -1].reshape(-1, 1)*ohca_reguli_inverse

In [181]:
mae = np.abs(y_head_test-y_test)
ans_mae = mae.sum()/mae.shape[0]

print('MAE of test set= ',ans_mae)

# 計算殘差變異
ss_residual = np.sum((y_test - y_head_test) ** 2)

# 計算總變異量
ss_total = np.sum((y_test - np.mean(y_test)) ** 2)

# 計算 R²
r_squared = 1 - (ss_residual / ss_total)

n = mae.shape[0]          # Number of data points
p = train_spatial_data.shape[1]            # Number of predictors

# Adjusted R-squared calculation
r_squared_adj = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)

print("R² of test set= ", r_squared)
print("ADJ R² of test set= ", r_squared_adj)

MAE of test set=  12.03608315984408
R² of test set=  0.21561557132228648
ADJ R² of test set=  5.314114357727425


# SHAP

In [182]:
# 假設 `spatial_data` 包含背景數據，用於 SHAP 的解釋
background_data = torch.FloatTensor(train_spatial_data[:, :-1])  
test_data = torch.FloatTensor(test_spatial_data[:, :-1])  

In [183]:
# 建立 SHAP 解釋器，使用背景數據
explainer = shap.GradientExplainer(s_net, background_data)
shap_values_test = explainer.shap_values(test_data)*ohca_reguli_inverse
# Get the shap values from my test data

test_features_df = h3_spatial_data.iloc[:, :-1]
feature_names = test_features_df.columns

In [184]:
feature_names

Index(['school', 'parking', 'grave_yard', 'college', 'hospital', 'fuel',
       'place_of_worship', 'fast_food', 'stage', 'restaurant', 'university',
       'theatre', 'library', 'post_office', 'pharmacy', 'fire_station',
       'toilets', 'childcare', 'car_wash', 'bank', 'ice_cream',
       'community_centre', 'cafe', 'shelter', 'ranger_station', 'shower',
       'veterinary', 'pub', 'cinema', 'animal_shelter', 'clinic',
       'exhibition_centre', 'nightclub', 'arts_centre', 'kindergarten',
       'bus_station', 'townhall', 'prison', 'doctors', 'dentist', 'studio',
       'marketplace', 'car_rental', 'driving_school', 'payment_centre', 'dojo',
       'gambling', 'bar', 'bicycle_parking', 'bbq', 'social_facility'],
      dtype='object')

In [185]:
#把SHAP 換成壞圖之格式
shap_col = shap_values_test.shape[0]
shap_row = shap_values_test.shape[1]
shap_values_test_2D = shap_values_test.reshape(shap_col,shap_row)
# shap.summary_plot(shap_values_test_2D, test_data,feature_names)

In [186]:
feature_names_w_SHAP = [f'shap {col}' for col in feature_names] # 在每個列名前加上 'shap'
SHAP_df = pd.DataFrame(shap_values_test_2D, columns=feature_names_w_SHAP) #換成 DF

In [187]:
df1 = test_h3_l7_df.reset_index(drop=True)
df2 = SHAP_df.reset_index(drop=True)
test_h3_l7_df_S = pd.concat([df1, df2], axis=1) #合併SHAP值到test_h3_l7_df

In [188]:
feature_names.shape[0]+3

54

In [189]:
# 初始化結果 DataFrame
spatial_data_score = pd.DataFrame()
spatial_data_score['id'] = test_h3_l7_df_S['id']

# 循環處理每一列
for col in range(feature_names.shape[0]):
    col_result = []  # 用於存儲當前列的計算結果
    
    # 遍歷每一行
    for row in range(test_h3_l7_df_S.shape[0]):
        denominator = test_h3_l7_df_S.iloc[row, col + 1]  # 分母
        numerator = test_h3_l7_df_S.iloc[row, col + feature_names.shape[0] + 3]  # 分子
        
        # 如果分母為 0，直接使用原分子數據
        if denominator == 0:
            col_result.append(numerator)
        else:
            col_result.append(numerator / denominator)  # 正常執行除法
    
    # 將當前列結果存入結果 DataFrame
    spatial_data_score[feature_names[col]] = col_result


In [190]:
spatial_data_score

,id,school,parking,grave_yard,college,hospital,fuel,place_of_worship,fast_food,stage,...,marketplace,car_rental,driving_school,payment_centre,dojo,gambling,bar,bicycle_parking,bbq,social_facility
0,872af0514ffffff,-0.894921,0.540944,0.062220,-0.092044,-0.207141,-0.476182,-0.418702,-0.958030,-0.044491,...,0.000000e+00,0.000000e+00,-0.081572,-0.014726,-0.084059,0.000000e+00,-0.044523,0.000000e+00,0.000000e+00,0.000000e+00
1,872af620cffffff,0.980137,0.222366,0.222447,0.064072,-0.221359,2.857026,0.672799,2.730987,-0.069770,...,0.000000e+00,0.000000e+00,-0.098371,0.003077,-0.168118,0.000000e+00,-0.069827,0.000000e+00,0.000000e+00,0.000000e+00
2,872af620dffffff,0.588646,-0.586764,0.118525,-0.039028,-0.324057,-0.872911,-0.616518,-1.523140,-0.066759,...,0.000000e+00,0.000000e+00,0.000000,0.000000,-0.168441,0.000000e+00,-0.066808,0.000000e+00,0.000000e+00,0.000000e+00
3,872af6220ffffff,-1.096774,0.016762,0.033712,0.009412,-0.183189,-0.802062,-0.460457,-1.941415,-0.111228,...,0.000000e+00,0.000000e+00,0.000000,-0.007260,-0.122736,0.000000e+00,-0.111309,0.000000e+00,0.000000e+00,0.000000e+00
4,872af6221ffffff,2.543599,0.245728,0.136710,0.038759,-0.287095,-1.047105,-1.149422,-0.570357,-0.092668,...,0.000000e+00,0.000000e+00,-0.228489,-0.065776,-0.080793,0.000000e+00,-0.092746,0.000000e+00,0.000000e+00,0.000000e+00
5,872af6224ffffff,-1.209560,0.132417,0.038228,-0.026053,-0.104429,-0.791445,-0.716325,-1.401340,0.000000,...,0.000000e+00,0.000000e+00,-0.072839,0.000000,-0.038676,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
6,872af6225ffffff,0.479245,1.153817,0.123048,0.057580,-0.056933,2.391667,1.586219,2.176862,-0.021619,...,0.000000e+00,0.000000e+00,-0.024763,0.006796,-0.121642,0.000000e+00,-0.021636,0.000000e+00,0.000000e+00,0.000000e+00
7,872af6228ffffff,2.650716,0.164806,0.133924,0.056869,-0.233537,2.549205,-0.304945,3.704378,-0.022333,...,0.000000e+00,0.000000e+00,-0.129812,-0.002059,-0.187392,0.000000e+00,-0.022349,0.000000e+00,0.000000e+00,0.000000e+00
8,872af6229ffffff,-0.726309,0.109039,0.000000,0.109117,-0.245742,1.095396,1.159591,1.193179,-0.088140,...,0.000000e+00,0.000000e+00,-0.058637,0.012450,0.000000,0.000000e+00,-0.088206,0.000000e+00,0.000000e+00,0.000000e+00
9,872af622affffff,-1.938079,0.042027,0.021633,-0.164219,-0.183189,-1.258287,-0.659522,-2.163155,-0.025685,...,0.000000e+00,0.000000e+00,-0.122015,-0.005742,-0.211130,0.000000e+00,-0.025703,0.000000e+00,0.000000e+00,0.000000e+00


In [191]:
# building_df = pd.read_csv('OpenStreetMap/B_features.csv')
poi_df = pd.read_csv('OpenStreetMap/A_features.csv')
# poi_df = poi_df[['osmid', 'amenity', 'ele', 'denomination', 'brand', 'cuisine',
#                     'healthcare', 'parking', 'geometry', 'height']
poi_df = poi_df[['osmid', 'amenity', 'ele', 'geometry']]

# 初始化列表
lat = []
lon = []
h3_l7 = []

# 只挑選有效的行
valid_indices = []  # 用來記錄有效行的索引
# 遍歷每一行
for i in range(poi_df.shape[0]):
    # 提取經緯度
    coordinates = re.findall("\d+\.?\d*", poi_df.geometry[i])
    if len(coordinates) < 3:
        continue  # 如果座標數據不完整，跳過該行
    else:
        # 提取緯度和經度
        latitude = float(coordinates[1])  # 緯度
        longitude = -float(coordinates[0])  # 經度
        lat.append(latitude)
        lon.append(longitude)
        
        # 轉換為 H3 L7 id
        h3_l7.append(h3.geo_to_h3(latitude, longitude, resolution=7))
        
        # 記錄有效行的索引
        valid_indices.append(i)

# 使用 valid_indices 創建新的 DataFrame
poi_df_valid = poi_df.iloc[valid_indices]

# 新增列到有效的 DataFrame
poi_df_valid['lat'] = lat
poi_df_valid['lon'] = lon
poi_df_valid['h3_l7'] = h3_l7

# 刪除 'geometry' 列
poi_df_valid = poi_df_valid.drop('geometry', axis=1)

<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Yuan\AppData\Local\Temp\ipykernel_14748\3698595432.py:17: SyntaxWarning: invalid escape sequence '\d'
  coordinates = re.findall("\d+\.?\d*", poi_df.geometry[i])
C:\Users\Yuan\AppData\Local\Temp\ipykernel_14748\3698595432.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi_df_valid['lat'] = lat
C:\Users\Yuan\AppData\Local\Temp\ipykernel_14748\3698595432.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [192]:
poi_df_valid

,osmid,amenity,ele,lat,lon,h3_l7
1001,37918264,school,3.0,36.850349,-75.986790,872af636cffffff
1002,38028116,school,7.0,36.897203,-76.142903,872af6309ffffff
1003,38028120,school,6.0,36.899387,-76.147563,872af6309ffffff
1004,38054683,school,3.0,36.841675,-75.979026,872af636dffffff
1005,38054722,parking,NaN,36.848191,-76.034993,872af636affffff
...,...,...,...,...,...,...
3478,13825469,place_of_worship,NaN,36.853923,-76.153865,872af6353ffffff
3479,14020613,place_of_worship,3.0,36.757505,-76.050832,872af0c92ffffff
3480,14325039,clinic,NaN,36.872663,-76.139490,872af6350ffffff
3481,14756225,parking,NaN,36.857801,-76.026476,872af636effffff


In [205]:
selected_values = spatial_data_score['id']
test_poi_df = poi_df_valid[poi_df_valid['h3_l7'].isin(selected_values)]

In [ ]:

test_poi_df = test_poi_df.reset_index(drop=True)
test_poi_df['score'] = 12  # 初始化 'score' 列為空值

# 循環處理每一行
for i in range(0, test_poi_df.shape[0]):
    poi_id = test_poi_df['h3_l7'].iloc[i]  # 取得當前行的 poi_id
    building_type = test_poi_df['amenity'].iloc[i]  # 取得當前行的 building_type
    
    # 查找 spatial_data_score 中對應的 id 和 amenity
    positions = spatial_data_score.index[spatial_data_score['id'] == poi_id]
 
        # 檢查 building_type 是否在 spatial_data_score 的列中
    if building_type in spatial_data_score.columns:
        building_score = spatial_data_score.loc[positions, building_type]
        
        if not building_score.empty:
            # 如果找到了對應的建築分數，將其轉換為數字並儲存
            test_poi_df.loc[i, 'score'] = pd.to_numeric(building_score.iloc[0])
        else:
            # 如果沒有找到對應的 building_type，可以設為 NaN 或其他預設值
            test_poi_df.loc[i, 'score'] = None
    else:
        # 如果 building_type 不存在於 spatial_data_score，設為 NaN 或其他預設值
        test_poi_df.loc[i, 'score'] = None


# 檢查結果
test_poi_df

C:\Users\Yuan\AppData\Local\Temp\ipykernel_14748\1704070939.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.4437302428054018' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_poi_df.loc[i, 'score'] = pd.to_numeric(building_score.iloc[0])


In [ ]:
# test_poi_df.to_csv('test_poi_df.csv', index=False, sep=',', encoding='utf-8-sig')

In [195]:
def intersection_area(r1, r2, d):
    if d >= r1 + r2:
        return 0  # 兩圓不相交
    elif d <= abs(r1 - r2):
        return np.pi * min(r1, r2)**2  # 一圓包含另一圓
    else:
        # 使用公式計算相交面積
        term1 = r1**2 * np.arccos((d**2 + r1**2 - r2**2) / (2 * d * r1))
        term2 = r2**2 * np.arccos((d**2 + r2**2 - r1**2) / (2 * d * r2))
        term3 = 0.5 * np.sqrt((-d + r1 + r2) * (d + r1 - r2) * (d - r1 + r2) * (d + r1 + r2))
        return term1 + term2 - term3


In [196]:
import math

#緯度（latitude） 經度（longitude）
def haversine(lat1, lon1, lat2, lon2):
    # 將經緯度轉換為弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # 計算差異
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # 哈弗辛公式
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # 地球半徑（公里）
    R = 6371.0
    
    # 計算距離
    distance = R * c #單位為km
    return distance 


In [197]:
test_poi_df

,osmid,amenity,ele,lat,lon,h3_l7,score
0,38028116,school,7.0,36.897203,-76.142903,872af6309ffffff,0.443730
1,38028120,school,6.0,36.899387,-76.147563,872af6309ffffff,0.443730
2,47294248,school,3.0,36.774569,-76.122297,872af6266ffffff,2.306914
3,47294251,school,2.0,36.767230,-76.108290,872af6260ffffff,4.607736
4,47298168,school,7.0,36.863398,-76.161385,872af6352ffffff,3.062870
...,...,...,...,...,...,...,...
902,11576328,university,NaN,36.803052,-76.187295,872af6276ffffff,-1.694479
903,11674543,library,NaN,36.844097,-76.116303,872af635cffffff,8.166481
904,13732541,fuel,NaN,36.822698,-76.126031,872af6358ffffff,1.484537
905,13825469,place_of_worship,NaN,36.853923,-76.153865,872af6353ffffff,0.359727


In [208]:
def area_score(AED_location, AED_range):
    score_list = []
    center_radius = 1.21  # 根據 H3 Level 7 的網格大小進行調整
    for i in range(0, test_poi_df.shape[0]):
        distance = haversine(AED_location[0], AED_location[1], test_poi_df.iloc[i]['lat'], test_poi_df.iloc[i]['lon'])
        
        # 如果 POI 在 AED 範圍外，跳過該 POI
        if distance > AED_range or test_poi_df.iloc[i]['score'] ==None:
            continue
        else:
            # 將 H3 網格編碼轉換為經緯度
            L7_center = h3.h3_to_geo(test_poi_df.iloc[i]['h3_l7'])
            center_distance = haversine(AED_location[0], AED_location[1], L7_center[0], L7_center[1])
            
            # 計算重疊面積的比例
            intersection_area_value = intersection_area(AED_range, center_radius, center_distance)
            proportion = intersection_area_value / ((center_radius ** 2) * np.pi)
            # 根據比例計算加權分數
            Score = test_poi_df.iloc[i]['score'] * proportion
            score_list.append(Score)
    
    return sum(score_list)  # 返回分數列表，方便後續處理


In [209]:
area_score(h3.h3_to_geo('872af6309ffffff'),1)

18.739343660654008

In [200]:
h3.h3_to_geo('872af6309ffffff')

(36.90609575498717, -76.14199701723689)

In [201]:
# ohca_df = pd.read_csv('OHCAs.csv')
# h3_l7 = []

# for i in range(ohca_df.shape[0]):
#     h3_l7.append(h3.geo_to_h3(ohca_df.Latitude[i], ohca_df.Longitude[i], resolution=7))

# ohca_df['h3_l7'] = h3_l7

# if min_lat < min(ohca_df['Latitude']): min_lat = min(ohca_df['Latitude'])
# if max_lat > max(ohca_df['Latitude']): max_lat = max(ohca_df['Latitude'])
# if min_lon < min(ohca_df['Longitude']): min_lon = min(ohca_df['Longitude'])
# if max_lon > max(ohca_df['Longitude']): max_lon = max(ohca_df['Longitude'])

# ohca_df = ohca_df.drop_duplicates(subset=['ReceivedTime', 'Latitude', 'Longitude'])
# ohca_df['ReceivedTime'] = pd.to_datetime(ohca_df['ReceivedTime'])
# ohca_df['ReceivedTime'] = ohca_df['ReceivedTime'].apply(lambda x: x.date())

In [202]:
# cols = np.concatenate((
#             poi_df.amenity.unique(),
#         ))
# len(cols)

# h3_l7_df = pd.DataFrame(data={'id': np.unique(np.concatenate((poi_df.h3_l7.unique(), ohca_df.h3_l7.unique())))})
# h3_l7_df[cols] = 0

# for i in range(poi_df.shape[0]):
#     h3_l7_id = poi_df.iloc[i]['h3_l7']
#     h3_l7_df.loc[h3_l7_df['id'] == h3_l7_id, poi_df.iloc[i]['amenity']] += 1
